# Windows 下使用 Yudao

## 开始前的准备

需要定义一些必须用到的 python 函数，以及 cmd 命令。

### 定义执行 cmd 的函数

In [ ]:
# 实现一个函数，用于执行 cmd 命令
import subprocess

def exe_cmd(cmd, show_output=False):
    try:
        output = subprocess.check_output(cmd, shell=True, text=True)
        if show_output == True:
            print(output)
        return output
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        return e

# exe_cmd("python --version", show_output=True)

ENV_COMMAND_VERSIONS = [
    "python --version ",
    "conda  --version ",
    "pip    --version ",
    "node   --version ",
    "npx    --version ",
    "npm    --version ",
    "dotnet --version ",
    "java   --version ",
]

for cmd in ENV_COMMAND_VERSIONS:
    output = exe_cmd(cmd)
    if isinstance(output, str):
        output = output.replace('/n', '/t')
    print(f"{cmd}: {output}")



python --version : Python 3.12.9	
conda  --version : conda 23.9.0	
pip    --version : pip 25.0.1 from C:\Users\zhong\anaconda3\envs\dify\Lib\site-packages\pip (python 3.12)		
node   --version : v22.17.0	
npx    --version : 11.4.2	
npm    --version : 11.4.2	
dotnet --version : 9.0.300	
java   --version : openjdk 18.0.2 2022-07-19	OpenJDK Runtime Environment (build 18.0.2+9-61)	OpenJDK 64-Bit Server VM (build 18.0.2+9-61, mixed mode, sharing)	


### 安装 uvx

uvx 是 uv 提供的命令行工具，用于快速运行 Python 脚本。

```
curl -LsSf https://astral.sh/uv/install.sh | sh
powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"
```

In [ ]:
cmd = 'powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"'
# exe_cmd(cmd, show_output=True)
cmd = "uvx --version"
print(f"{cmd}: {exe_cmd(cmd)}")


### 安装 postgresql

我是在我的 dragonai.vip 服务器上安装 postgresql 的。

安装后需要修改 postgresql 的配置文件，将监听地址从 localhost 改为 0.0.0.0，允许远程连接。

修改配置文件的方法如下：

1. 打开 postgresql 的配置文件，一般在 `/etc/postgresql/14/main/postgresql.conf` 中。
2. 找到 `listen_addresses` 配置项，将其值改为 `0.0.0.0`。
3. 保存配置文件并重启 postgresql 服务。

```bash
sudo service postgresql restart
``` 
修改后，需要修改 postgresql 的 pg_hba.conf 文件，将 localhost 改为 0.0.0.0，允许远程连接。

修改 pg_hba.conf 文件的方法如下：

1. 打开 pg_hba.conf 文件，一般在 `/etc/postgresql/14/main/pg_hba.conf` 中。
2. 找到 `host` 配置项，将其值改为 `0.0.0.0/0`。
3. 保存配置文件并重启 postgresql 服务。

```bash
sudo service postgresql restart
``` 

最后修改 yudao 中 application-local.yaml 文件，将数据库连接地址改为 postgresql.dragonai.vip。

```yaml
spring:
  datasource:
    url: jdbc:postgresql://postgresql.dragonai.vip:5432/ruoyi-vue-pro # PostgreSQL 连接的示例
    username: postgres
    password: dragonai2025!
    # url: jdbc:oracle:thin:@127.0.0.1:1521:xe # Oracle 连接的示例
``` 

### 配置 yudao 数据库访问权限

默认的账号因为不是 postgres 管理员，可能没有数据库的修改权限，可参考如下命令进行设置。

``` sql
-- 查看是否已经打勾（有权限）
SELECT has_schema_privilege('yudao', 'public', 'CREATE');

-- 授予USAGE权限（访问模式中的对象）
GRANT USAGE ON SCHEMA public TO yudao;

-- 授予CREATE权限（允许创建表）
GRANT CREATE ON SCHEMA public TO yudao;

-- 可选：授予所有权限（包括修改、删除等）
-- GRANT ALL PRIVILEGES ON SCHEMA public TO yudao;

-- 查看配置结果
SELECT has_schema_privilege('yudao', 'public', 'CREATE');
```

### 初始化数据库数据

因为我用的是 postgresql，所以使用了默认的数据库表生成脚本 $/sql/postgresql 下的 sql 文件对数据库进行初始化构建。

其中有两个文件:
1. quartz.sql
2. ruoyi-vue-pro.sql

### 配置 postgresql 驱动

因为 yudao 默认使用的是 mysql，当使用 postgresql 时，需要配置一下启用 postgresql jdbc驱动。
修改 maven 依赖中的配置：yudao-framework/yudao-spring-boot-starter-mybatis/pom.xml。

去掉 optional 配置:

``` XML
<dependency>
    <groupId>org.postgresql</groupId>
    <artifactId>postgresql</artifactId>
    <!-- <optional>true</optional> -->
</dependency>
```

### 安装启动 Redis

安装略。

启动略。

## 启动 yudao 程序

完成数据库配置、初始化，启动了 redis 后，就可以启动 yudao 程序了。

### 启动 Debug 模式

```
C:\Users\zhong\git\ruoyi-vue-pro> C: && cd C:\Users\zhong\git\ruoyi-vue-pro && cmd /C "Z:\Service\Java\jdk-18.0.2\bin\java.exe -agentlib:jdwp=transport=dt_socket,server=n,suspend=y,address=localhost:60324 @C:\Users\zhong\AppData\Local\Temp\cp_b9cyrfkm7nr7gxgfypuqnfbxn.argfile cn.iocoder.yudao.server.YudaoServerApplication "
```




## 启用 BPM 模块

修改 pom.xml ，启用 yudao-module-bpm 模块：

```xml
<module>yudao-module-bpm</module>
```

修改 yudao-server\pom.xml ，添加 yudao-module-bpm 模块的依赖：

```xml
<dependency>
    <groupId>cn.iocoder.boot</groupId>
    <artifactId>yudao-module-bpm</artifactId>
    <version>${revision}</version>
</dependency>
```
初始化 BPM 模块的数据库表：
1. mysql: yudao-module-bpm\src\test\resources\sql\create_tables.sql
2. postgresql: yudao-module-bpm\src\test\resources\sql\create_tables_pgsql.sql


配置启用 yudao-module-bpm 之后，需要重新 build yudao-server 模块。

```bash
cd yudao-server
mvn clean install -U
```
